In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
from tqdm import tqdm
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset, RandomSampler, SequentialSampler
import pickle
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from ast import literal_eval

import nltk
import re
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
import pandas as pd
data_path = '/content/drive/My Drive/DL_Data'

# EDA for czEng dataset

In [0]:
cz1 = pd.read_csv(os.path.join(data_path, "czeng20-train"), sep="\t", header=None, quoting=3)
cz2 = pd.read_csv(os.path.join(data_path, "czeng20-test"), sep="\t", header=None, quoting=3)

In [0]:
cz1 = cz1[[4,5]]
cz1.columns = ['Czech','English']
cz2 = cz2[[4,5]]
cz2.columns = ['Czech','English']

In [206]:
cz_data = pd.concat([cz1, cz2],ignore_index=True, sort =False)

,Czech,English
0,"Ty, drsný dítě z Kwangtungu?",A tough kid from Kwangtung?
1,"Dítě, jehož předkové viseli v koších nad propa...",A kid whose ancestors used to hang in baskets ...
2,"Chci po tobě jen, aby ses proměnil v mouchu na...",All I'm asking you to do is be a fly on the wall.
3,Tvý předci se teď obrací v hrobech.,Your ancestors are rolling in their graves.
4,Moc na mě tlaçíte.,You're pushing too much.
...,...,...
492591,Ale tentokrát... se musíme vkrást do Linderman...,Only this time... we have to sneak into Linder...
492592,"Přiznej si to, Hiro.","Face it, Hiro."
492593,Je to ztracený případ.,It's a lost cause.
492594,"Já se nikdy nevzdám, Ando.",Ando. I will never give up.


# EDA for OdEng dataset

In [0]:
od1 = pd.read_csv(os.path.join(data_path, "train.final"), sep="\t", header=None)
od1.columns = ['Source', 'English', 'Odia']

od2 = pd.read_csv(os.path.join(data_path, "dev.final"), sep="\t", header=None)
od2.columns = ['Source', 'English', 'Odia']

od3 = pd.read_csv(os.path.join(data_path, "test.final"), sep="\t", header=None)
od3.columns = ['Source', 'English', 'Odia']

In [0]:
od_data = pd.concat([od1, od2, od3], ignore_index=True)

In [108]:
od_data

,Source,English,Odia
0,genesis,In the beginning God created the heaven and th...,ଆରମ୍ଭରେ ପରମେଶ୍ବର ଆକାଶ ଓ ପୃଥିବୀକୁ ସୃଷ୍ଟି କଲେ।
1,genesis,"And the earth was without form, and void; and ...",ପୃଥିବୀ ସେତବେେଳେ ସଂପୂରନ୍ଭାବେ ଶୂନ୍ଯ ଓ କିଛି ନଥିଲା...
2,genesis,"And God saw the light, that it was good: and G...","ପରମେଶ୍ବର ଆଲୋକକୁ ଦେଖିଲେ ଏବଂ ସେ ଜାଣିଲେ, ତାହା ଉତ୍..."
3,genesis,"And God called the light Day, and the darkness...","ପରମେଶ୍ବର ସହେି ଆଲୋକର ନାମ ଦେଲେ "" ଦିନ"" ଏବଂ ଅନ୍ଧକା..."
4,genesis,"And God said, Let there be a firmament in the ...","ଏହାପରେ ପରମେଶ୍ବର କହିଲେ, ""ଜଳ ମଧିଅରେ ବୃହତ ଗମ୍ବୁଜ ..."
...,...,...,...
25831,odia_bibhaba,But he is best remembered for his numerous sto...,ଫତୁରାନନ୍ଦଙ୍କର ଯେ କୌଣସି ରଚନା ପାଠ କଲେ ବି ତାଙ୍କର ...
25832,odia_bibhaba,He was deeply influenced by a senior writer an...,ପ୍ରଥମରୁ ସେ ଲେଖକ ହେବା କଥା ଭାବି ନ ଥିଲେ। ପରେ କାନ୍...
25833,odia_bibhaba,"Towards the end of his life, he was not keepin...",ଜୀବନର ଶେଷ ବେଳକୁ ତାଙ୍କ ଶରୀର ସୁସ୍ଥ ରହୁ ନ ଥିଲା। ତ...
25834,odia_bibhaba,Having carved out a permanent place for himsel...,"ତାଙ୍କ ରଚନା ଭିତରେ ରହିଛି ଗଳ୍ପ, ଉପନ୍ୟାସ, କାବ୍ୟ, କ..."


# EDA for psEng dataset
## kde4 dataset

In [0]:
kde4 = pd.read_csv(os.path.join(data_path, "KDE4.en-ps.en"), sep='\n', header=None)
kde4.columns = ['English']
kde4.insert(0, 'Source', 'KDE4')
kde4['Pashto'] = pd.read_csv(os.path.join(data_path, "KDE4.en-ps.ps"),sep='\n', header=None)

In [66]:
kde4

,Source,English,Pashto
0,KDE4,Size & & Orientation,کچ & & لورموندنه
1,KDE4,Graphics Adaptor,کښنيز اړوونی
2,KDE4,3D Options,غوراوي 3D
3,KDE4,Monitor Gamma,ليداني ګاما
4,KDE4,Multiple Monitors,ګڼ ليداني
...,...,...,...
3372,KDE4,An error occurred while loading %1:,سپړاويز ونه کوت
3373,KDE4,Error:,پټې دوتنې ښودل
3374,KDE4,Tag Resource,د ځای چلونې لپاره کېکاږﺉ
3375,KDE4,Please select the file to open.,چلول


## Ubuntu dataset

In [0]:
ubuntu = pd.read_csv(os.path.join(data_path, "Ubuntu.en-ps.en"), sep='\n', header=None)
ubuntu.columns = ['English']
ubuntu.insert(0, 'Source', 'Ubuntu')
ubuntu['Pashto'] = pd.read_csv(os.path.join(data_path, "Ubuntu.en-ps.ps"),sep='\n', header=None)

In [68]:
# Drop the first 53 rows since it has English in 'Pashto' language
ubuntu_clean = ubuntu[54:]
ubuntu_clean

,Source,English,Pashto
54,Ubuntu,Selected Link,ټاکل شوې تړنه
55,Ubuntu,Specifies whether the AtkHyperlink object is s...,څيز ټاکل شوی وي Atk-ځانګړې کوي چې که چېرې لوپېوند
56,Ubuntu,Number of Anchors,د استلو شمېر
57,Ubuntu,The number of anchors associated with the AtkH...,څيز پورې نښتي وي Atk-د هغو استلو شمېر چې د لوپ...
58,Ubuntu,End index,پای لړيال
...,...,...,...
9640,Ubuntu,Show warning options,خبرتيا غوراوي ښودل
9641,Ubuntu,Scale options,کچ غوراوي
9642,Ubuntu,Show scale options,کچ غوراوي ښودل
9643,Ubuntu,Miscellaneous options,ګډوډ غوراوي


## Ted talk dataset

In [0]:
ted = pd.read_csv(os.path.join(data_path, "ted-wmt20.en-ps.en"), sep='\n',header=None, quoting=3)
ted.columns = ['English']
ted.insert(0, 'Source', 'Ted')
ted['Pashto'] = pd.read_csv(os.path.join(data_path, "ted-wmt20.en-ps.ps"),sep='\n', header=None)

In [83]:
ted

,Source,English,Pashto
0,Ted,"A public, Dewey long ago observed, is constitu...",کلونه مخکې، ډیوي ،امریکایي فیلسوف) وپتیېله چې) .
1,Ted,If we are to call the tyranny of assumptions i...,یوه ټولنه د بحث او مناظرې په ترڅ کې رامنځته کې...
2,Ted,It is in this spirit that I join into a discus...,تر بحث او مناظرې لاندې راولو د همدې روحیې په ل...
3,Ted,To put the assumptions very clearly: capitalis...,د دولت جوړونې د پروژې لپار بسیج او تنظیم شي :د...
4,Ted,If we looked in the world of 1945 and looked a...,او په همدې توګه دموکراسي که چېرې د ۱۹۴۵ نړۍ ته...
...,...,...,...
659,Ted,"Ladies and gentlemen, if we glance to human hi...",که موږ انساني تاريخ و ګورو د ښځې داستان د ښځې ...
660,Ted,"Ladies and gentlemen, this plight of millions ...",يوه شاعره وايي: نجلۍ ودېږي، مجبورېږي او بيا ډې...
661,Ted,"A few days after Malala was born, my daughter ...",د ميلينونو مېرمنو دا بد حالت بدلېداى شي که موږ...
662,Ted,You put your life at stake for the cause of tr...,دې سمدستي راته وويل مهرباني وکه ته ولې ځان ملا...


# Bible dataset

In [0]:
bible = pd.read_csv(os.path.join(data_path, "bible.en-ps.clean.en"), sep='\n', header=None, quoting=3)
bible.columns = ['English']
bible.insert(0, 'Source', 'Bible')
bible['Pashto'] = pd.read_csv(os.path.join(data_path, "bible.en-ps.clean.ps"),sep='\n', header=None)

In [128]:
bible['English'][3]

'God saw that the light was good. God separated the light from the darkness.'

In [123]:
bible['Pashto'][3]

'خُدائ پاک په دې ليدو خوشحاله وو . بيا هغۀ رڼا د تيارې نه جُدا کړه ،'

In [124]:
bible['English'][4]

'God called the light day. He called the darkness night. There was evening and there was morning, one day.'

In [125]:
bible['Pashto'][4]

'اؤ په رڼا يې ” ورځ “ نُوم کېښودو اؤ په تيارۀ يې ” شپه “ . شپه تېره شوه . اؤ سحر شو . دا وړومبۍ ورځ وه .'

In [129]:
bible['English'][5]

'Then God said: "Let there be an expanse in the middle of the waters. Let it separate the waters from the waters."'

In [131]:
bible['Pashto'][5]

'بيا خُدائ پاک حُکم وکړو چې ، ” د پاسنو او لاندينو اوبو تر مينځه دې فضا جوړه شى . “'

### Combine all Odi datasets 

In [0]:
ps_data = pd.concat([kde4, ubuntu, ted, bible], ignore_index=True, sort =False)

In [89]:
ps_data

,Source,English,Pashto
0,KDE4,Size & & Orientation,کچ & & لورموندنه
1,KDE4,Graphics Adaptor,کښنيز اړوونی
2,KDE4,3D Options,غوراوي 3D
3,KDE4,Monitor Gamma,ليداني ګاما
4,KDE4,Multiple Monitors,ګڼ ليداني
...,...,...,...
27113,Bible,"The Spirit and the bride say: "" Come "" And let...",رُوحُ القُدس اَؤ ناوے وائى چه ” راشه ! “ اَؤ ه...
27114,Bible,I testify to every man who hears the words of ...,زَۀ هر چا ته دا خبردارے ورکوم چه دَ دے کتاب دَ...
27115,Bible,If any man shall take away from the words of t...,که څوک دَ پيشن ګويو دَ دے کتاب نه څۀ کمے وکړى ...
27116,Bible,"He who testifies to these things says: "" I com...",اَؤ څوک چه دَ دے خبرو شهادت ورکوى هغه وائى چه ...


-------------------------------------------------
# Preprocessing

In [0]:
# Module for creating dictionary and dataloaders

#Classes: Dictionary, TensoredDataset
#Fcns: indexize_dataset, pad_list_of_tensors, pad_collate_fn

# Dictionary object of all vocab in dataset to indices
class Dictionary(object):
    def __init__(self, datasets, language, include_valid=False):
        self.tokens = []
        self.ids = {}
        self.counts = {}
        
        # add special tokens
        self.add_token('<pad>')
        self.add_token('<unk>')
        
        for line in tqdm(datasets[language]):
            for w in line:
                self.add_token(w)
      
    def add_token(self, w):
        if w not in self.tokens:
            self.tokens.append(w)
            _w_id = len(self.tokens) - 1
            self.ids[w] = _w_id
            self.counts[w] = 1
        else:
            self.counts[w] += 1

    def get_id(self, w):
        return self.ids[w]
    
    def get_token(self, idx):
        return self.tokens[idx]
    
    def decode_idx_seq(self, l):
        return [self.tokens[i] for i in l]
    
    def encode_token_seq(self, l):
        return [self.ids[i] if i in self.ids else self.ids['<unk>'] for i in l]
    
    def __len__(self):
        return len(self.tokens)


#Encode all reviews in index form using above dictionary
def indexize_dataset(datasets, language, dictionary):
    indexized_datasets = []
    for l in tqdm(datasets[language]):
        encoded_l = dictionary.encode_token_seq(l)
        indexized_datasets.append(encoded_l)
        
    return indexized_datasets


#TensoredDataset object is needed to create Dataloaders for model training, uses the indexized datasets
class TensoredDataset(object):
    def __init__(self, list_of_lists_of_tokens):
        self.input_tensors = []
        self.flagged_index = []
        
        for sample in list_of_lists_of_tokens:
            self.input_tensors.append(torch.tensor([sample], dtype=torch.long))
        
    def __len__(self):
        return len(self.input_tensors)
    
    def __getitem__(self, idx):
        # return a (input, target) tuple
        return (self.input_tensors[idx])



def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = 40
    padded_list = []
    
    for t in list_of_tensors:    
        padded_tensor = torch.cat([t, torch.tensor([[pad_token]*(max_length - t.size(-1))], dtype=torch.long)], dim = -1)
        padded_list.append(padded_tensor[:max_length])
        
    padded_tensor = torch.cat(padded_list, dim=0)
    
    return padded_tensor

def pad_collate_fn(batch):
    # batch is a list of sample tuples
    token_list = [s[0] for s in batch]
    
    #pad_token = persona_dict.get_id('<pad>')
    pad_token = 0
    
    input_tensor = pad_list_of_tensors(token_list, pad_token)
    
    return input_tensor

# Random seed
seed = 1029

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

# Create Dictionary and Dataloader

### Train-Test split
train:70%, val:15%, test:15% 

stratified by source

In [0]:
od_train, remain_od = train_test_split(od_data, test_size=0.3, stratify=od_data['Source'])
od_val, od_test = train_test_split(remain_od, test_size=0.5, stratify=remain_od['Source'])

In [0]:
ps_train, remain_ps = train_test_split(ps_data, test_size=0.3, stratify=ps_data['Source'])
ps_val, ps_test = train_test_split(remain_ps, test_size=0.5, stratify=remain_ps['Source'])

### Make a dictionary

In [151]:
en_ps_dict = Dictionary(kde4, language='English', include_valid=False)
ps_dict = Dictionary(kde4, language='Pashto', include_valid=False)
#en_od_dict = Dictionary(od_data, language='English', include_valid=False)
#od_dict = Dictionary(od_data, language='Odia', include_valid=False)
#en_cz_dict = Dictionary(fr_data, language='English', include_valid=False)
#cz_dict = Dictionary(fr_data, language='French', include_valid=False)

100%|██████████| 3377/3377 [00:00<00:00, 59401.98it/s]


In [0]:
#save Dictinaries
pickle_dict = open(data_dir + "dictionary.p","wb")
pickle.dump(review_dict, pickle_dict)
pickle_dict.close()

In [163]:
# Indexited dataset
#indexized_od_train_dataset = indexize_dataset(od_train, od_dict)
indexized_ps_train_dataset = indexize_dataset(ps_train, language='Pashto', dictionary = ps_dict)
#indexized_cz_train_dataset = indexize_dataset(fr_train, fr_dict)

#indexized_od_val_dataset = indexize_dataset(od_val, od_dict)
indexized_ps_val_dataset = indexize_dataset(ps_val, language='Pashto',dictionary =ps_dict)
#indexized_cz_val_dataset = indexize_dataset(fr_val, fr_dict)

#indexized_od_test_dataset = indexize_dataset(od_test, od_dict)
indexized_ps_test_dataset = indexize_dataset(ps_test, language='Pashto', dictionary =ps_dict)
#indexized_cz_test_dataset = indexize_dataset(fr_test, fr_dict)

# English
#indexized_en_od_train_dataset = indexize_dataset(od_train, en_od_dict)
indexized_en_ps_train_dataset = indexize_dataset(ps_train, language='English', dictionary =en_ps_dict)
#indexized_en_cz_train_dataset = indexize_dataset(fr_train, en_fr_dict)

#indexized_en_od_val_dataset = indexize_dataset(od_val, en_od_dict)
indexized_en_ps_val_dataset = indexize_dataset(ps_val,language='English', dictionary =en_ps_dict)
#indexized_en_cz_val_dataset = indexize_dataset(fr_val, en_fr_dict)

#indexized_en_od_test_dataset = indexize_dataset(od_test, en_od_dict)
indexized_en_ps_test_dataset = indexize_dataset(ps_test, language='English', dictionary =en_ps_dict)
#indexized_en_cz_test_dataset = indexize_dataset(fr_test, en_fr_dict)

100%|██████████| 4068/4068 [00:00<00:00, 85623.81it/s]


In [0]:
# Tensored dataset
#tensor_od_tr_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_ps_tr_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_cz_tr_dataset = TensoredDataset(indexized_fr_train_dataset)

#tensor_od_val_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_ps_val_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_cz_val_dataset = TensoredDataset(indexized_fr_train_dataset)

#tensor_od_test_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_ps_test_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_cz_test_dataset = TensoredDataset(indexized_fr_train_dataset)

# English
#tensor_en_od_tr_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_en_ps_tr_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_en_cz_tr_dataset = TensoredDataset(indexized_fr_train_dataset)

#tensor_en_od_val_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_en_ps_val_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_en_cz_val_dataset = TensoredDataset(indexized_fr_train_dataset)

#tensor_en_od_test_dataset = TensoredDataset(indexized_od_train_dataset)
tensor_en_ps_test_dataset = TensoredDataset(indexized_ps_train_dataset)
#tensor_en_cz_test_dataset = TensoredDataset(indexized_fr_train_dataset)

In [170]:
# check the first example
tensor_en_ps_val_dataset[100]

tensor([[ 38,   7,  27,   4,  24,  10,  11,   7,   6, 131]])

In [0]:
# Random seed
seed = 1029

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))


# Dataloader
batch_size = 32

#od_train_dataloader = DataLoader(tensor_od_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
ps_train_dataloader = DataLoader(tensor_ps_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_train_dataloader = DataLoader(tensor_fr_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

#od_val_dataloader = DataLoader(tensor_od_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
ps_val_dataloader = DataLoader(tensor_ps_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_val_dataloader = DataLoader(tensor_fr_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

#od_test_dataloader = DataLoader(tensor_od_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
ps_test_dataloader = DataLoader(tensor_ps_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_test_dataloader = DataLoader(tensor_fr_tset_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

# English
#en_od_train_dataloader = DataLoader(tensor_od_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
en_ps_train_dataloader = DataLoader(tensor_ps_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_fr_train_dataloader = DataLoader(tensor_fr_tr_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

#en_od_val_dataloader = DataLoader(tensor_od_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
en_ps_val_dataloader = DataLoader(tensor_ps_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_fr_val_dataloader = DataLoader(tensor_fr_val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

#en_od_test_dataloader = DataLoader(tensor_od_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
en_ps_test_dataloader = DataLoader(tensor_ps_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)
#cz_fr_test_dataloader = DataLoader(tensor_fr_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=pad_collate_fn, worker_init_fn=_init_fn)

## Save Dataloaders

In [0]:
#ps_test_dataloader = open(os.path.join(data_dir, "ps_test_dataloader.p"),"wb")
#pickle.dump(ps_test_dataloader, ps_test_dataloader)
#ps_test_dataloader.close()